# 1. ABOUT HYPERPARAMETERS
Out of the three approaches to hyperparameter tuning — manual, machine-assisted, and algorithmic — this tutorial will focus on machine-assisted hyperparameter tuning. 

**Learning rate**
Learning rate controls how much to update the weight in the optimization algorithm. We can use a fixed learning rate, a gradually decreasing learning rate, momentum-based methods or adaptive learning rates. This depends on our choice of optimizer such as SGD, Adam, Adagrad, AdaDelta or RMSProp. The **mother of all hyperparameters**, the learning rate, quantifies the learning progress of a model in a way that can be used to optimize its capacity.

**Convolution Kernel Width** 
In Convolutional Neural Networks (CNNs), the Kernel Width influences the number of parameters in a model which, in turns, influences its capacity.

**Momentum** 
Learns the direction of the next step using the knowledge of the previous steps. It helps to prevent oscillations.

**Number of epochs**
Number of epochs is the the number of times the entire training set passes through the neural network. One should increase the number of epochs until there is small gap between the test error and the training error.

**Batch size**
Mini batch size is the number of sub samples given to the network after which the parameter update happens. 

**Activation function**
The activation funtion introduces non-linearity to the model. Usually, rectifier works well. Other alternatives are sigmoid, tanh, and other activation functions depening on the task.

**Number of hidden layers and units**
The number of hidden units is key to regulate the representational capacity of a model. Add more layers until the test error no longer improves, however computationally expensive to train the network. Having a small amount of units may lead to underfitting while having more units are usually not harmful with appropriate regularization. 

**Weight initialization**
We should initialize the weights with small random numbers to prevent dead neurons, but not too small to avoid zero gradient. Uniform distribution usually works well.

**Dropout for regularization**
Dropout is a preferable regularization technique to avoid overfitting in deep neural networks. The method simply drops out units in neural network according to the desired probability. A default value of 0.5 is a good choice to test with.

## 1.1 Overview
Below is a list of the topics we are going to cover:

### Part 1: Grid Search Hyperparameter Tuning
- How to use Keras models in scikit-learn.
- How to use grid search in scikit-learn.
- How to tune: 
    - batch size and training epochs
    - optimization algorithms
    - learning rate and momentum.
    - network weight initialization
    - activation functions
    - dropout regularization
    - number of neurons in the hidden layer.
    
### Part 2: Bayesian Optimization for Hyperparameter Tuning
- How to use Bayesian optimizations on hyperparameters of a Gradient Boosting Machine (lightgbm)
- How to use the Hyperopt library for Bayesian optimization
- How to compare Random Search to Bayesian optimization for hyperparameter tuning
- How to visualize Bayesian vs Random search optimization

## 1.2 How to Use Keras Models in scikit-learn
Keras models can be used in scikit-learn by wrapping them with the KerasClassifier or KerasRegressor class.

To use these wrappers you must define a function that creates and returns your Keras sequential model, then pass this function to the build_fn argument when constructing the KerasClassifier class. For example:

In [2]:
def model():
    """A whole bunch of model junk"""
    return model

# model = KerasClassifier(build_fn=create_model)
model = KerasRegressor(
    build_fn=model, 
    nb_epoch=300, 
    batch_size=30,
    verbose=1
)

# 2. HYPERPARAMETER TUNING

Manually tuning hyperparameters impractical. There are two generic approaches to sampling search candidates. 

**Random search** samples a given number of candidates from a parameter space with a specified distribution. Although **grid search** is useful in many machine learning algoirthms, it **is not efficient** in tuning hyperparamters for deep neural networks because, as the number of parameters increases, computation grows exponentially. **Random search has been found more efficient compared to grid search** in hyperparameter tuning for deep neural network (see “Random Search for Hyper-Parameter Optimization”). 

**Grid search** is a model hyperparameter optimization technique that exhaustively searches all parameter combinations for given values. In scikit-learn this technique is provided in the GridSearchCV class. When constructing this class you must provide a dictionary of hyperparameters to evaluate in the **param_grid** argument. 

- param_grid = dict(epochs=[10,20,30])
- grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
- grid_result = grid.fit(X, y)

To implement grid search more efficiently, start with broad ranges of hyperparameter values. Consider performing a coarse grid search for smaller number of epochs or smaller training set. At the next stage, perform a narrower search with more epochs or a larger part of the training set.

By default, accuracy is the score that is optimized, but other scores can be specified in the score argument of the GridSearchCV constructor.

By default, the grid search will only use one thread思路. By setting the n_jobs argument in the GridSearchCV constructor to -1, the process will use all cores on your machine. Depending on your Keras backend, this may interfere with the main neural network training process.

The GridSearchCV process will then construct and evaluate one model for each combination of parameters. Cross validation is used to evaluate each individual model and the default of 3-fold cross validation is used, although this can be overridden by specifying the cv argument to the GridSearchCV constructor. Once completed, you can access the outcome of the grid search in the result object returned from grid.fit(). The best_score_ member provides access to the best score observed during the optimization procedure and the best_params_ describes the combination of parameters that achieved the best results.

## Install/Import Python Packages

In [1]:
#Standard packages
import pandas as pd
import numpy as np
import seaborn as sns

#Machine learning packages
from sklearn.svm import SVC, SVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid, cross_val_score, cross_val_predict, train_test_split, KFold, StratifiedKFold
from sklearn import datasets, linear_model, metrics, preprocessing, pipeline
from sklearn.pipeline import make_union, make_pipeline, Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression, LassoCV, RidgeCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, ExtraTreesRegressor, BaggingClassifier, BaggingRegressor, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor, VotingClassifier
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, accuracy_score, r2_score, log_loss, make_scorer, get_scorer, f1_score, auc, roc_auc_score, roc_curve, precision_score, recall_score, precision_recall_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib
import xgboost
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
#Deep Learning packages
import tensorflow
import keras
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.datasets import make_classification, make_circles
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.initializers import RandomUniform

#Plotting and Display
import matplotlib.pyplot as plt
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

#Reproducibility
np.random.seed(42)

/opt/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Using TensorFlow backend.


## 2.1 Grid Search, Right Quick

### 2.1.1 Generate Target And Feature Data

In [4]:
# Number of features
number_of_features = 100

# Generate features matrix and target vector
features, target = make_classification(n_samples = 10000,
                                       n_features = number_of_features,
                                       n_informative = 3,
                                       n_redundant = 0,
                                       n_classes = 2,
                                       weights = [.5, .5],
                                       random_state = 0)

### 2.1.2 Create a Function That Constructs A Neural Network

In [8]:
# Create function returning a compiled network
def create_network(optimizer='rmsprop'):
    
    # Start neural network
    network = Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(Dense(units=16, activation='relu', input_shape=(number_of_features,)))

    # Add fully connected layer with a ReLU activation function
    network.add(Dense(units=16, activation='relu'))

    # Add fully connected layer with a sigmoid activation function
    network.add(Dense(units=1, activation='sigmoid'))

    # Compile neural network
    network.compile(loss='binary_crossentropy', # Cross-entropy
                    optimizer=optimizer, # Optimizer
                    metrics=['accuracy']) # Accuracy performance metric
    
    # Return compiled network
    return network

### 2.1.3 Wrap Function In KerasClassifier

In [9]:
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, verbose=0)

### 2.1.4 Create Hyperparameter Search Space

In [10]:
# Create hyperparameter space
epochs = [5, 10]
batches = [5, 10, 100]#8,12,16
optimizers = ['rmsprop', 'adam']

# Create hyperparameter options
hyperparameters = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)

### 2.1.5 Conduct Grid Search

In [11]:
# Create grid search
grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters)

# Fit grid search
grid_result = grid.fit(features, target)

e:\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


### 2.1.6 Find Best Model’s Hyperparameters

In [12]:
# View hyperparameters of best neural network
grid_result.best_params_

{'batch_size': 10, 'epochs': 5, 'optimizer': 'adam'}

In [14]:
grid_result.best_score_

0.9321999923586846

# Tuning Process

In [2]:
df_APEC_total = pd.read_excel('./data/df_APEC_total.xlsx',index_col=0).fillna(0)
data = df_APEC_total
train_dataset = data.sample(frac=0.8,random_state=42)
test_dataset = data.drop(train_dataset.index)

train_stats = train_dataset.describe()
train_stats = train_stats.drop(['Part weight','Length','Angle', 'Height', 'Width'], axis=1)
train_stats = train_stats.transpose()
train_labels = train_dataset[['Part weight']]
test_labels = test_dataset[['Part weight']]

def norm(x):
    return (x - train_stats['mean']) / train_stats['std'] #zero-mean normalization, same with sklearn.preprocessing.StandardScaler
train_dataset = train_dataset.drop(['Part weight','Length','Angle','Height', 'Width'], axis=1)
test_dataset = test_dataset.drop(['Part weight','Length','Angle','Height', 'Width'], axis=1)
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

X_train = normed_train_data.fillna(0)
X_test = normed_test_data.fillna(0)
y_train = train_labels.values
y_test = test_labels.values

x = np.array(X_train, dtype=np.float)
y = np.array(y_train, dtype=np.float)
X_train = x.astype(float)
y_train = y.astype(float)
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
mol_num, feat_num = X_train.shape
print("Number of materials for training = %i, Number of features = %i\n" % (mol_num, feat_num))
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

Number of materials for training = 801, Number of features = 50



## 2.2 Detailed Hyperparameter Tuning

### 2.2.1 Tune Batch Size and Number of Epochs
The batch size in iterative gradient descent is the number of patterns shown to the network before the weights are updated. It is also an optimization in the training of the network, defining how many patterns to read at a time and keep in memory.

The number of epochs is the number of times that the entire training dataset is shown to the network during training. Some networks are sensitive to the batch size, such as LSTM recurrent neural networks and Convolutional Neural Networks. We will evaluate a suite of different mini batch sizes from 10 to 100 in steps of 20.

**Result**: We can see that the batch size of 20 and 100 epochs achieved the best result of about 68% accuracy.

In [3]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import backend as K

def create_model():
    net = Sequential()
    net.add(Dense(50, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(25, kernel_initializer='normal', activation='relu'))
    net.add(Dense(10, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal'))
    # compile this model
    net.compile(loss='mean_absolute_error', # one may use 'mean_absolute_error' as alternative
                  optimizer='adam',
                  metrics=[r2_keras] # you can add several if needed
                 )
    return net

# create model
model = KerasRegressor(
    build_fn=create_model, 
    verbose=0
)

# define the grid search parameters
batch_size = [10, 50, 100, 150, 200]
epochs = [300]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2',cv=2)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.912470 using {'batch_size': 150, 'epochs': 300}
0.876630 (0.015176) with: {'batch_size': 10, 'epochs': 300}
0.865758 (0.071278) with: {'batch_size': 50, 'epochs': 300}
0.747206 (0.158582) with: {'batch_size': 100, 'epochs': 300}
0.912470 (0.001007) with: {'batch_size': 150, 'epochs': 300}
0.896324 (0.019961) with: {'batch_size': 200, 'epochs': 300}


### 2.2.2 Tune the Training Optimization Algorithm
Keras offers a suite of different state-of-the-art optimization algorithms.
Here, we tune the optimization algorithm used to train the network, each with default parameters.

In [4]:
def create_model(optimizer = 'Adam'):
    net = Sequential()
    net.add(Dense(50, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(25, kernel_initializer='normal', activation='relu'))
    net.add(Dense(10, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal'))
    
    # compile this model
    net.compile(loss='mean_absolute_error',
                  optimizer=optimizer,
                  metrics=[r2_keras]
                 )

    return net

# create model
model = KerasRegressor(build_fn=create_model, epochs=300, batch_size=150, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,scoring='r2',cv=2)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.919902 using {'optimizer': 'Adam'}
-38.144256 (1.779182) with: {'optimizer': 'SGD'}
0.480141 (0.026124) with: {'optimizer': 'RMSprop'}
-1691.879831 (148.768536) with: {'optimizer': 'Adagrad'}
-2626.417978 (61.124426) with: {'optimizer': 'Adadelta'}
0.919902 (0.013206) with: {'optimizer': 'Adam'}
0.909336 (0.013474) with: {'optimizer': 'Adamax'}
0.793662 (0.010069) with: {'optimizer': 'Nadam'}


### 2.2.1 Tune Learning Rate and Momentum
Pre-select an optimization algorithm to train your network and tune its parameters. The most common optimization algorithm is Stochastic Gradient Descent (SGD). Here, we optimize the SGD learning rate and momentum parameters.

Learning rate controls how much to update the weight at the end of each batch. The momentum controls how much to let the previous update influence the current weight update.

First, we will try a range of small standard learning rates and a momentum values from 0.2 to 0.8 in steps of 0.2 and 0.9.

Include the number of epochs in an optimization like this because there is a dependency between the amount of learning per batch (learning rate), the number of updates per epoch (batch size) and the number of epochs.

In [6]:
from tensorflow.keras.optimizers import Adam
def create_model(learn_rate=0.01):
    net = Sequential()
    net.add(Dense(50, input_dim=feat_num, kernel_initializer='normal', activation='relu'))
    net.add(Dense(25, kernel_initializer='normal', activation='relu'))
    net.add(Dense(10, kernel_initializer='normal', activation='relu'))
    net.add(Dense(1, kernel_initializer='normal'))
    optimizer = Adam(lr=learn_rate)
    # compile this model
    net.compile(loss='mean_absolute_error',
                  optimizer=optimizer,
                  metrics=[r2_keras]
                 )
    return net

# create model
model = KerasRegressor(build_fn=create_model, epochs=300, batch_size=150, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]

param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,scoring='r2',cv=2)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.915163 using {'learn_rate': 0.001}
0.915163 (0.002663) with: {'learn_rate': 0.001}
0.797625 (0.006874) with: {'learn_rate': 0.01}
-5.682463 (4.140839) with: {'learn_rate': 0.1}
-6.028000 (1.308491) with: {'learn_rate': 0.2}
-0.009291 (0.009272) with: {'learn_rate': 0.3}


### 2.2.1 Tune Network Weight Initialization
We tune the selection of network weight initialization by evaluating all of the available techniques.

We will use the same weight initialization method on each layer. It may be better to use different weight initialization schemes according to the activation function used on each layer. In the example below we use rectifier for the hidden layer. We use sigmoid for the output layer because the predictions are binary.

In [8]:
def create_model(init_mode='normal'):
    net = Sequential()
    net.add(Dense(50, input_dim=feat_num, kernel_initializer=init_mode, activation='relu'))
    net.add(Dense(25, kernel_initializer=init_mode, activation='relu'))
    net.add(Dense(10, kernel_initializer=init_mode, activation='relu'))
    net.add(Dense(1, kernel_initializer=init_mode))
    # compile this model
    net.compile(loss='mean_absolute_error', 
                  optimizer='adam',
                  metrics=[r2_keras]
                 )
    return net

# create model
model = KerasRegressor(build_fn=create_model, epochs=300, batch_size=150, verbose=0)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,scoring='r2',cv=2)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.949018 using {'init_mode': 'glorot_uniform'}
0.914193 (0.007762) with: {'init_mode': 'uniform'}
0.937584 (0.008559) with: {'init_mode': 'lecun_uniform'}
0.931289 (0.006615) with: {'init_mode': 'normal'}
-2549.191762 (59.352398) with: {'init_mode': 'zero'}
0.935371 (0.004504) with: {'init_mode': 'glorot_normal'}
0.949018 (0.001734) with: {'init_mode': 'glorot_uniform'}
0.942498 (0.003234) with: {'init_mode': 'he_normal'}
0.948605 (0.007488) with: {'init_mode': 'he_uniform'}


### 2.2.1 Tune the Neuron Activation Function
The activation function controls the non-linearity of individual neurons and when to fire.

Generally, the rectifier activation function is the most popular, but it used to be the sigmoid and the tanh functions and these functions may still be more suitable for different problems.

In this example, we will evaluate the suite of different activation functions available in Keras. We will only use these functions in the hidden layer, as we require a sigmoid activation function in the output for the binary classification problem.

Generally, it is a good idea to prepare data to the range of the different transfer functions, which we will not do in this case.

In [9]:
def create_model(activation='relu'):
    net = Sequential()
    net.add(Dense(50, input_dim=feat_num, kernel_initializer='glorot_uniform', activation=activation))
    net.add(Dense(25, kernel_initializer='glorot_uniform', activation=activation))
    net.add(Dense(10, kernel_initializer='glorot_uniform', activation=activation))
    net.add(Dense(1, kernel_initializer='glorot_uniform'))
    # compile this model
    net.compile(loss='mean_absolute_error', 
                  optimizer='adam',
                  metrics=[r2_keras]
                 )
    return net

# create model
model = KerasRegressor(build_fn=create_model, epochs=300, batch_size=150, verbose=0)


# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,scoring='r2',cv=2)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.946451 using {'activation': 'softplus'}
-2403.637573 (62.577298) with: {'activation': 'softmax'}
0.946451 (0.001707) with: {'activation': 'softplus'}
-1510.902820 (92.599545) with: {'activation': 'softsign'}
0.936178 (0.009662) with: {'activation': 'relu'}
-1446.382356 (57.987487) with: {'activation': 'tanh'}
-1705.264945 (160.803576) with: {'activation': 'sigmoid'}
-2096.795019 (14.886832) with: {'activation': 'hard_sigmoid'}
0.835198 (0.029097) with: {'activation': 'linear'}


### 2.2.1 Tune Dropout Regularization
In this example, we will look at tuning the dropout rate for regularization in an effort to limit overfitting and improve the model’s ability to generalize.

To get good results, dropout is best combined with a weight constraint such as the max norm constraint.
This involves fitting both the dropout percentage and the weight constraint. We will try dropout percentages between 0.0 and 0.9 (1.0 does not make sense) and maxnorm weight constraint values between 0 and 5.

In [11]:
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.constraints import maxnorm

def create_model(dropout_rate=0.0, weight_constraint=0):
    net = Sequential()
    net.add(Dense(50, input_dim=feat_num, kernel_initializer='glorot_uniform', activation='softplus', kernel_constraint=maxnorm(weight_constraint)))
    net.add(Dropout(dropout_rate))
    net.add(Dense(25, kernel_initializer='glorot_uniform', activation='softplus'))
    net.add(Dropout(dropout_rate))
    net.add(Dense(10, kernel_initializer='glorot_uniform', activation='softplus'))
    net.add(Dropout(dropout_rate))
    net.add(Dense(1, kernel_initializer='glorot_uniform'))
    # compile this model
    net.compile(loss='mean_absolute_error', 
                  optimizer='adam',
                  metrics=[r2_keras]
                 )
    return net

# create model
model = KerasRegressor(build_fn=create_model, epochs=300, batch_size=150, verbose=0)

# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,scoring='r2',cv=2)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/opt/miniconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.963363 using {'dropout_rate': 0.0, 'weight_constraint': 4}
0.955086 (0.008820) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.960956 (0.001886) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.953183 (0.006294) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.963363 (0.006454) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.960561 (0.001802) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
-17.648935 (2.320878) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
-8.331620 (4.779226) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
-3.432321 (0.305546) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
-3.046291 (0.790028) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
-8.288844 (3.023236) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
-10.571475 (1.299413) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
-12.361010 (1.181767) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
-9.697801 (0.580042) with: {'dropout_rate': 0.2, 'weight_co

### 2.2.1 Tune the Number of Neurons in the Hidden Layer

The number of neurons in a layer is an important parameter to tune. Generally the number of neurons in a layer controls the representational capacity of the network, at least at that point in the topology.

Also, generally, a large enough single layer network can approximate any other neural network, at least in theory.

In this example, we will look at tuning the number of neurons in a single hidden layer. We will try values from 1 to 30 in steps of 5.

A larger network requires more training and at least the batch size and number of epochs should ideally be optimized with the number of neurons.
**Result** We can see that the best results were achieved with a network with 5 neurons in the hidden layer with an accuracy of about 71%

In [13]:
def create_model(neurons=50):
    net = Sequential()
    net.add(Dense(neurons, input_dim=feat_num, kernel_initializer='glorot_uniform', activation='softplus', kernel_constraint=maxnorm(4)))
    net.add(Dense(25, kernel_initializer='glorot_uniform', activation='softplus'))
    net.add(Dense(10, kernel_initializer='glorot_uniform', activation='softplus'))
    net.add(Dense(1, kernel_initializer='glorot_uniform'))
    # compile this model
    net.compile(loss='mean_absolute_error', 
                  optimizer='adam',
                  metrics=[r2_keras]
                 )
    return net
# create model
model = KerasRegressor(build_fn=create_model, epochs=300, batch_size=150, verbose=0)

# define the grid search parameters
neurons = [25, 50, 75, 100, 200]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,scoring='r2',cv=2)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.962870 using {'neurons': 100}
0.946851 (0.003272) with: {'neurons': 25}
0.952416 (0.005549) with: {'neurons': 50}
0.955074 (0.000973) with: {'neurons': 75}
0.962870 (0.003316) with: {'neurons': 100}
0.957021 (0.003070) with: {'neurons': 200}


## 2.3.1 Tips for Hyperparameter Optimization


**k-fold Cross Validation** You can see that the results from the examples show some variance. A default cross-validation of 3 was used, but perhaps k=5 or k=10 would be more stable. Carefully choose your cross validation configuration to ensure your results are stable.

**Review the Whole Grid.** Do not just focus on the best result, review the whole grid of results and look for trends to support configuration decisions.

**Parallelize.** Use all your cores if you can, neural networks are slow to train and we often want to try a lot of different parameters. Consider spinning up a lot of AWS instances.
Use a Sample of Your Dataset. Because networks are slow to train, try training them on a smaller sample of your training dataset, just to get an idea of general directions of parameters rather than optimal configurations.

**Start with Coarse Grids.** Start with coarse-grained grids and zoom into finer grained grids once you can narrow the scope.

**Do not Transfer Results.** Results are generally problem specific. Try to avoid favorite configurations on each new problem that you see. It is unlikely that optimal results you discover on one problem will transfer to your next project. Instead look for broader trends like number of layers or relationships between parameters.

**Reproducibility is a Problem.** Although we set the seed for the random number generator in NumPy, the results are not 100% reproducible. There is more to reproducibility when grid searching wrapped Keras models than is presented here.